In [1]:
import pandas as pd
import numpy as np
import requests
from pyquery import PyQuery as pq
import re
import json

# 2019+2020

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
#
url_2019 = 'http://web.archive.org/web/20191203024617/https://www.kobo.com/tw/zh/p/tw-kobo-2019top100'
url_2020 = 'https://www.kobo.com/tw/zh/p/tw-kobo-2020top100'
urls = {2019: url_2019, 2020: url_2020}

In [3]:
if False:
    for y, url in urls.items():
        r = requests.get(url, headers=headers)
        r.encoding = 'utf-8'
        doc = pq(r.text)
        #
        books_data = []
        #
        books = doc.find('div.item.book.basic-item')
        for idx, book in enumerate(books):
            book = pq(book)
            #
            title = book.find('p.title.product-field').eq(0).text()
            author = book.find('span.contributor-name').eq(0).text()
            try:
                star = book.find('div.kobo.star-rating').eq(0).attr('aria-label')
                star = float(star.replace(' out of 5 stars', '').replace('Rated ', ''))
            except Exception as err:
                star = 0
            price = int(book.find('p.product-field.price > span > span').eq(0).text().replace('NT$', ''))
            #
            if y == 2019:
                src = book.find('img.cover-image').eq(0).attr('src').replace('//web.archive.org/web/20191203024617im_/', '')
            else:
                src = book.find('img.cover-image').eq(0).attr('data-lazy')
            src = ('https:' not in src) and 'https:'+src or src
            #
            href = book.find('a.carousel-tab-navigation-target').eq(0).attr('href').replace('/web/20191203024617/', '')
            href = ('https:' not in href) and 'https://www.kobo.com'+href or href
            #
            rank = idx+1
            #
            data = {
                'title': title,
                'author': author,
                'star': star,
                'price': price,
                'src': src,
                'href': href,
                'rank': rank,
            }
            books_data.append(data)
        #
        df_books = pd.DataFrame(books_data).assign(year=y)
        df_books.to_csv(f'kobo100_{y}.csv', index=False)
        print(f'儲存kobo100_{y}.csv')

# 2019

In [4]:
# url_2019 = 'https://www.kobo.com/tw/zh/list/kobo-2019/cz57jFime8-QbQjXQANmxQ?pageNumber='

In [5]:
# books_data_2019 = []
# for page in range(1, 5):
#     url = url_2019+str(page)
#     r = requests.get(url, headers=headers)
#     r.encoding = 'utf-8'
#     doc = pq(r.text)
#     #
#     books = doc.find('div.item-detail')
#     for book in books:
#         book = pq(book)
#         #
#         title = book.find('p.title.product-field').eq(0).text()
#         author = book.find('span.visible-contributors a').eq(0).text()
#         tmp = book.find('div.kobo.star-rating').eq(0).attr('aria-label')
#         star = float(re.search('^Rated (.+?) out of.+$', tmp).group(1))
#         price = int(book.find('p.product-field.price > span > span').eq(0).text().replace('NT$', ''))
#         #
#         src = book.find('img.cover-image').eq(0).attr('src')
#         src = ('https:' not in src) and 'https:'+src or src
#         #
#         href = book.find('p.title.product-field a').eq(0).attr('href')
#         href = ('https:' not in href) and 'https://www.kobo.com'+href or href
#         #
#         data = {
#             'title': title,
#             'author': author,
#             'star': star,
#             'price': price,
#             'src': src,
#             'href': href,
#             # 'rank': rank,
#         }
#         books_data_2019.append(data)
# #         print(href)

In [6]:
# df_books_2019 = pd.DataFrame(books_data_2019).reset_index().rename(columns={'index': 'rank'})
# df_books_2019['rank'] = df_books_2019['rank'] + 1
# df_books_2019 = df_books_2019.reindex(columns=list(df_books_2019.columns[1:]) + ['rank']).assign(year='2019')
# df_books_2019

In [7]:
# df_books_2019.to_csv('kobo100_2019.csv', index=False)

In [8]:
# df_all = df_books.merge(df_books_2019[['title', 'rank']], on='title', how='left')
# df_all.rename(columns={'rank_x': '2020_rank', 'rank_y': '2019_rank'}, inplace=True)
# df_all.drop(columns='year', inplace=True)
# #
# df_all = df_all.fillna(0).astype({'2019_rank': 'int'})
# #
# df_all

# JSON

In [9]:
# import pandas as pd
# import numpy as np
# import requests
# from pyquery import PyQuery as pq
# import re
# import json

In [10]:
n = 100
df_2019 = pd.read_csv('kobo100_2019.csv').reset_index().rename(columns={'index': 'idx'})[:n]
df_2020 = pd.read_csv('kobo100_2020.csv').reset_index().rename(columns={'index': 'idx'})[:n]
#
df_all = df_2019.merge(df_2020, on='title', how='outer')
df_all = df_all.fillna(300).astype({'rank_x': int, 'rank_y': int, 'idx_x': int, 'idx_y': int})
#
df_all[:3]

,idx_x,title,author_x,star_x,price_x,src_x,href_x,rank_x,year_x,idx_y,author_y,star_y,price_y,src_y,href_y,rank_y,year_y
0,0,原子習慣,詹姆斯．克利爾,4.5,196.0,https://kbimages1-a.akamaihd.net/92214e93-fa53...,https://www.kobo.com/tw/zh/ebook/pGo8y2e4ET6pd...,1,2019.0,0,詹姆斯．克利爾,4.5,182.0,https://kbimages1-a.akamaihd.net/92214e93-fa53...,https://www.kobo.com/tw/zh/ebook/pGo8y2e4ET6pd...,1,2020.0
1,1,真確：扭轉十大直覺偏誤，發現事情比你想的美好,漢斯．羅斯林,4.5,238.0,https://kbimages1-a.akamaihd.net/787950cf-36ca...,https://www.kobo.com/tw/zh/ebook/tG1aCXbY1zG4O...,2,2019.0,5,漢斯．羅斯林,4.5,221.0,https://kbimages1-a.akamaihd.net/787950cf-36ca...,https://www.kobo.com/tw/zh/ebook/tG1aCXbY1zG4O...,6,2020.0
2,2,人生複本【燒腦神作，零負評！】,布萊克．克勞奇,4.5,232.0,https://kbimages1-a.akamaihd.net/cbcfd485-ac07...,https://www.kobo.com/tw/zh/ebook/ejIfELnxhDyrt...,3,2019.0,8,布萊克．克勞奇,4.5,216.0,https://kbimages1-a.akamaihd.net/cbcfd485-ac07...,https://www.kobo.com/tw/zh/ebook/ejIfELnxhDyrt...,9,2020.0


In [11]:
where = (df_all['idx_y'] == 300) #| (df_all['idx_x'] == 300)
newN = df_all[where].shape[0]

In [12]:
cols_map = {
    "idx": "node",
    "title": "name",
}
#
# df_2019_dict = df_2019.rename(columns=cols_map)[cols_map.values()].to_dict('records')
df_2019_dict = df_2019.rename(columns=cols_map).to_dict('records')
#
df_2020['idx'] = df_2020['idx'] + 100
# df_2020_dict = df_2020.rename(columns=cols_map)[cols_map.values()].to_dict('records')
df_2020_dict = df_2020.rename(columns=cols_map).to_dict('records')

In [13]:
nodes = df_2019_dict+df_2020_dict+[{'node': 200, 'name': "退榜"}, {'node': 201, 'name': "新進"}]
# nodes[8]

In [14]:
m = len(nodes)
m

202

In [15]:
df_tmp = df_all[['idx_x', 'idx_y']]
default = 3
df_tmp['value'] = default
#
where = df_tmp['idx_y'] < 300
df_tmp['idx_y'] = np.where(where, df_tmp['idx_y'] + 100, df_tmp['idx_y'])
# 2020 new
where = df_tmp['idx_x'] == 300
df_tmp['idx_x'] = np.where(where, m-1, df_tmp['idx_x'])
df_tmp['value'] = np.where(where, default*2/newN, df_tmp['value'] )
# 2019 stop
where = df_tmp['idx_y'] == 300
df_tmp['idx_y'] = np.where(where, m-2, df_tmp['idx_y'])
df_tmp['value'] = np.where(where, default*2/newN, df_tmp['value'] )
#

df_tmp.rename(columns={'idx_x': 'source', 'idx_y': 'target'}, inplace=True)
#
df_tmp[:3]

/home/pan/anaconda3/envs/django_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/pan/anaconda3/envs/django_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pan/anaconda3/envs/django_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,source,target,value
0,0,100,3.0
1,1,105,3.0
2,2,108,3.0


In [16]:
tmp1 = {'source': 0, 'target': m-1, 'value': 0}
tmp2 = {'source': m-2, 'target': m-3, 'value': 0}
links = df_tmp.to_dict('records') + [tmp1, tmp2]
links[:3]

[{'source': 0, 'target': 100, 'value': 3.0},
 {'source': 1, 'target': 105, 'value': 3.0},
 {'source': 2, 'target': 108, 'value': 3.0}]

In [17]:
data = {
    'nodes': nodes,
    'links': links,
}
with open('kobo_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

In [18]:
len(links)

167